In [ ]:
import pythonmlp # https://github.com/florisvanvugt/PythonMLP
import random
import scipy.stats
from tqdm import tqdm
import statsmodels.stats.proportion
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:


def f(x):
    return x

interact(f, x=10);

In [ ]:
truth_s = .1 # as before

truth_m = 50
truth_a = .1

def answer(x):
    p = pythonmlp.pyes(x,truth_a,truth_m,truth_s)
    return(np.random.uniform()<p) # transform into true or false

N = 30
for stim in [0,50,100,200]:
    resps = [ answer(stim) for _ in range(N) ]
    print("stimulus {} : proportion yes {:.2f}".format(stim,np.mean(resps)))

In [ ]:
def simulate_block(answer_func,MAXSTIM=200,NHYP=200,SLOPE_HYP=.1,
                   NTRIALS=30,N_CATCH_TRIALS=6):
        
    mlp = pythonmlp.MLP(
    
        # The slope of our psychometric curves
        slope = SLOPE_HYP, # this is a cheat, we give the real psychometric curve slope...
        
        # The minimum and maximum of the hypothesised thresholds
        hyp_min = 0,
        hyp_max = MAXSTIM,
        
        # The number of hypotheses
        hyp_n = NHYP,
        
        # Our false alarm rates (these will be crossed with the threshold hypotheses)
        fa = [0.,.1,.2,.3,.4],
    )
    
        
    # Initialise a list that includes the "catch" trials,
    # so we plan in advance that we have the right number of catch trials.
    trialsA = [ "mlp" for _ in range(NTRIALS-1) ] + [ "catch" for _ in range(N_CATCH_TRIALS) ]
    #trialsB = [ "mlp" for _ in range(NTRIALS_B)   ] + [ "catch" for _ in range(N_CATCH_TRIALS_B) ]
    random.shuffle(trialsA)
    #random.shuffle(trialsB)
    
    # The trials we want to do
    todo = ["mlp"] + trialsA #+ trialsB # this is done so that the first trial is never a catch
    
    # Now let's simulate these trials
    
    stim = MAXSTIM # start at the maximum level
    
    trials = []
    for trial,info in enumerate(todo):
            
        stim = stim if stim>0 else 0 # set to 0 if lower
        stim = 0 if info=='catch' else stim
        
        ans = answer_func(stim)
        mlp.update(stim,ans)
        trials.append({
            "trial":trial+1,
            "kind":info,
            "stimulus":stim,
            "response":ans
            })
        stim = mlp.next_stimulus()


    m = mlp.get_midpoint_estimate()
    
    res = {"estimate_m":m}
           
    return res

In [ ]:
def do_sim(
    MAXHYP,
    NHYP,
    HYP_SLOPE,
    MIDPOINT_M,
    MIDPOINT_SD,
    TRUTH_A,
    TRUTH_S,
    N_PARTICIPANTS,
    N_BLOCKS,
    N_TRIALS,
    N_CATCH_TRIALS,
    ):
    
    truths  = np.random.normal(MIDPOINT_M,MIDPOINT_SD,N_PARTICIPANTS)
    
    results = []
    for midpoint in tqdm(truths):
        
        t = {}
        t['true.midpoint']=midpoint
        
        # Now simulate and estimate
        
        estimates = []
        def answ(stim):
            p = pythonmlp.pyes(stim,TRUTH_A,t['true.midpoint'],TRUTH_S)
            return(np.random.uniform()<p) # transform into true or false
        for j in range(N_BLOCKS): # three blocks
            res = simulate_block(answ,
                                 MAXHYP,NHYP,HYP_SLOPE,
                                 N_TRIALS,N_CATCH_TRIALS)
            estimates.append(res['estimate_m'])
        t['midpoint.estimated']=np.mean(estimates)
    
        results.append(t)
    results = pd.DataFrame(results)
    
    results['mismatch'] = results['midpoint.estimated']-results['true.midpoint']

    print("Simulated {} participants".format(N_PARTICIPANTS))
    print("Mean midpoint mismatch = {:.3f}  SD = {:.3f}".
          format(
              np.mean(results['mismatch']),
              np.std(results['mismatch'])))
    print("Mean absolute midpoint mismatch = {:.3f}  SD = {:.3f}".
          format(
              np.mean(np.abs(results['mismatch'])),
              np.std(np.abs(results['mismatch']))))
    r = np.corrcoef(results['midpoint.estimated'],results['true.midpoint'])
    print("True to estimated midpoint correlation : r={:.3f}".format(r[0,1]))

    f,ax = plt.subplots(1,1)
    plt.plot(results['true.midpoint'],results['midpoint.estimated'],'o')
    plt.xlabel("True curve midpoint")
    plt.ylabel("Estimated curve midpoint")
    plt.axline((0,0), slope=1., color='gray', alpha=.2, linestyle='--')
    ax.set_aspect('equal')

In [ ]:
do_sim(

    # Psychometric curve properties
    MAXHYP = 200,
    NHYP = 200,
    HYP_SLOPE = .1,
    
    # Psychometric curve midpoints: mean and spread
    MIDPOINT_M = 30,
    MIDPOINT_SD = 10,
    
    # Simulated "ground truth" psychometric curves
    TRUTH_A = .1, # real false alarm rate
    TRUTH_S = .1, # real slope parameter
    
    N_PARTICIPANTS = 10, # how many participants to simulate
    
    N_BLOCKS = 3, # how many blocks to test
    
    # Part 1 number of "real" trials (MLP)
    N_TRIALS        = 10,
    # Part 1 number of catch trials (zero stimulus)
    N_CATCH_TRIALS = 2,    
)

In [ ]:
interact_manual(
    do_sim,

    # Psychometric curve properties
    MAXHYP = widgets.IntSlider(min=0, max=500, step=1, value=200, tooltip='Psychometric curve hypotheses spread maximum'),
    NHYP = widgets.IntSlider(min=1, max=500, step=1, value=200, tooltip='Number of psychometric curve hypotheses'),
    HYP_SLOPE = widgets.FloatSlider(min=.001, max=5, step=.01, value=.1, tooltip='Slope parameter of psychometric curve'), # hypothesized slope
    
    # Psychometric curve midpoints: mean and spread
    MIDPOINT_M = widgets.FloatSlider(min=0, max=200, step=.1, value=30, tooltip='Ground truth psychometric curve midpoint mean (M)'),
    MIDPOINT_SD = widgets.FloatSlider(min=0, max=200, step=.1, value=10, tooltip='Ground truth psychometric curve midpoint spread (SD)'),
    
    # Simulated "ground truth" psychometric curves
    TRUTH_A = widgets.FloatSlider(min=0, max=1, step=.01, value=.1, tooltip='Ground truth false alarm rate'), # ground truth false alarm rate
    TRUTH_S = widgets.FloatSlider(min=0, max=10, step=.01, value=.1, tooltip='Ground truth psychometric curve slope'), # ground truth slope parameter
    
    N_PARTICIPANTS = widgets.IntSlider(min=0, max=100, step=1, value=50, tooltip='# of participants to simulate'), # how many participants to simulate
    
    N_BLOCKS = widgets.IntSlider(min=1, max=10, step=1, value=3, tooltip='# of blocks per participant'), # how many blocks to test per participant
    
    # Part 1 number of "real" trials (MLP)
    N_TRIALS        = widgets.IntSlider(min=0, max=100, step=1, value=30, tooltip='# of MLP trials per block per participant'),
    # Part 1 number of catch trials (zero stimulus)
    N_CATCH_TRIALS = widgets.IntSlider(min=0, max=100, step=1, value=6, tooltip='# of (additional) catch trials per block per participant'),
    
);

In [ ]:

# Some whitespace to prevent flickering from the code above...



















